
# <p style="text-align: center;"> *PyData-Atlanta* </p>

<p style="text-align: center;"> Visualizing Equations Notebook </p>
---
 
 

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from IPython.core.display import HTML, display
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
zeros4 = np.load('zeros4.npy')
zeros13 = np.load('zeros13.npy')
zeros16 = np.load('zeros16.npy')

In [ ]:
def make_df(degree):
    from itertools import product
    rootbound = int(sum(x * (2 ** (x -1)) for x in range(degree+1)))
    roots = np.zeros((rootbound,2))
    index = 0
    for j in range(degree):
        rootsj = np.load('zerosEqual' + str(j + 1) + '.npy')
        for z in rootsj:
            roots[index] = z
            index = index + 1
    df = pd.DataFrame(roots, columns = ['real', 'imaginary'])
    return df

In [ ]:
%time df23 = make_df(23)

In [ ]:
%time df25 = make_df(25)

For mapping a small number of roots, the default parameters look very good. For example:

In [ ]:
plt.xkcd()
plt.scatter(zeros4.real, zeros4.imag)
plt.ylabel('Imaginary')
plt.xlabel('Real')
plt.axis([-1.5,1.5,-1.5,1.5])
plt.title('zeros of 0,1 polynomials of degrees <= 4')

In [ ]:
plt.show()

Note how long it takes to display the next image.

In [ ]:
plt.xkcd()
plt.scatter(zeros16.real, zeros16.imag)
plt.ylabel('Imaginary')
plt.xlabel('Real')
plt.axis([-1.5,1.5,-1.5,1.5])
plt.title('zeros of 0,1 polynomials of degrees <= 16')

In [ ]:
%time plt.show()

In [ ]:
len(zeros16)

So, there are close to a million roots that we are trying to display.

To get an 'xkcd' version of the original:
- increase the figure size
- change the marker to a point
- decrease the marker size
- color it black



In [ ]:
plt.figure(figsize=(10,10))
plt.ylabel('y')
plt.xlabel('x')
plt.title('zeros of 0,1 polynomials of degrees <= 16')
plt.axis([-1.5,1.5,-1.5,1.5])
plt.scatter(zeros16.real, zeros16.imag, c = "black", s=0.010, marker = ".")

In [ ]:
%time plt.show()

## Bokeh

Bokeh has an interactive zoom feature.

In [ ]:
from bokeh.plotting import figure
from bokeh.io import output_notebook, reset_output, show
output_notebook()

We now consider roots of 0,1 polynomials of degree less than or equal to 13.

In [ ]:
to_plot = figure(title = 'zeros of 0,1 polynomials of degrees <= 13')
to_plot.circle(zeros13.real, zeros13.imag, color = "black",size = 0.1)

Visualize the precision of the solutions.

In [ ]:
show(to_plot)

## Datashader bokeh extension

Datashader takes as input pandas dataframes. With the Bokeh extension of datashader called **InteractiveImage** it is possible to use work interactively with millions of points, and to visualize on the order of a billion points.

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

In [ ]:
%%time
cvs = ds.Canvas(plot_width = 600, plot_height = 600, x_range=(-1.5,1.5), y_range=(-1.5,1.5))
agg = cvs.points(df25, 'real', 'imaginary')

In [ ]:
%time tf.shade(agg, cmap=["white", "black"])

In the paper, the authors mentioned that by plotting roots of randomly chosen 0,1 polynomials, it became clear that the fractal-like parts of the figure were due to very few polynomials.

In [ ]:
tf.shade(agg.where(agg <= 1), cmap=["white", "black"])

In [ ]:
tf.shade(agg.where(agg > 10000), cmap=["white", "black"])

In [ ]:
from datashader.colors import inferno, viridis
import colorcet as cc

In [ ]:
tf.shade(agg, cmap=viridis)

In [ ]:
tf.set_background(tf.shade(agg, cmap=cc.dimgray), "black")

In [ ]:
tf.set_background(tf.shade(agg, cmap=cc.fire), "black")

In [ ]:
export(tf.set_background(tf.shade(agg, cmap=cc.fire), "black"),"roots25")

# Datashader interactive zoom

In [ ]:
import bokeh.plotting as bp
from datashader.bokeh_ext import InteractiveImage

What did I mean by "fractal-like"?

In [ ]:
new_plot = bp.figure(tools='pan,wheel_zoom,box_zoom,reset', x_range=(-1.5,1.5), y_range=(-1.5,1.5))

def image_callback(x_range, y_range, w, h):
    cvs = ds.Canvas(plot_width = 600, plot_height = 600, x_range=x_range, y_range=y_range)
    agg = cvs.points(df23, 'real', 'imaginary', ds.any())
    img =tf.set_background(tf.shade(agg, cmap=cc.fire), "black")
    return tf.dynspread(img, max_px = 100) 

In [ ]:
InteractiveImage(new_plot, image_callback)

In [ ]:
%%time
def image_callback(x_range, y_range, w, h):
    cvs = ds.Canvas(plot_width = 600, plot_height = 600, x_range=x_range, y_range=y_range)
    agg = cvs.points(df23, 'real', 'imaginary')
    img =tf.set_background(tf.shade(agg, cmap=list(reversed(cc.fire))), "black")
    return tf.spread(img)

Don't forget to zoom on z=1. Up to degree 23 is as high as possible. Degree 24 has wait times after zooming of a minute.

In [ ]:
%time InteractiveImage(new_plot, image_callback)

J. Bednar: if your dataset has 1 million points, encoding that as JSON would result in an HTML file that would be unreadable with today's browsers, so datashader keeps that data stored efficiently in a Python process, ready for delivery as an image into the browser so that the browser only ever sees a fixed-size data structure. So you can either have standalone HTML files, or the ability to handle large datasets; you cannot have both (with today's browser technologies).

InteractiveImage is actually a tiny bit of code borrowed from HoloViews, and we've now added datashader support to the current Github master of holoviews (to be released as version 1.7). With holoviews you can easily choose between using matplotlib and using bokeh to view your plots, and the advantage of the matplotlib backend is that matplotlib figures can be export to svg or png easily. See an example of using holoviews with datashader here, where you can change 'bokeh' to 'matplotlib' to use matplotlib instead (and remove some of the axis=None lines to show the axes). See the Exporting tutorial at holoviews.org for options for exporting plots.

Zoom on imaginary number...note the patterns around it.
Past ten decimal places it is clear some of these points are a visualization of the error in root finding. Would be interesting to compare the numpy roots with the fortran roots, or write an extension module that interfaces to that fortran subroutine. 

Thank you!